In [ ]:
%pip install photutils

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
from astropy import coordinates as coords
from astropy import units as u
from numpy.lib.recfunctions import append_fields
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAperture
from photutils.aperture import SkyCircularAperture
from photutils.aperture import SkyEllipticalAperture
from regions import Regions,PixCoord
from astropy.coordinates import SkyCoord

In [ ]:
Ha_image = '/Users/jameswells/Documents/astro/datasets/ngc3741data/ngc3741Deepae_crop.fits'
hdu = fits.open(Ha_image)[0]
wcs = WCS(hdu.header)

regions = Regions.read('/Users/jameswells/Documents/astro/datasets/ngc3741data/regions/diffuse_h2_new.reg', format='ds9')
print(len(regions))
print(regions[0])
print(regions[0].center.ra.value)
print(regions[0].width)

In [ ]:
fig=plt.figure(figsize=(8,8))
ax1 = plt.subplot(projection=wcs)
ax1.set_facecolor('white')
ax1.set_xlabel('RA (2000)')
ax1.set_ylabel('Dec (2000)')
ra_ax1 = ax1.coords['ra']
dec_ax1 = ax1.coords['dec']
cmapa = plt.get_cmap('binary')
alpimg = ax1.imshow(hdu.data,origin='lower',cmap=plt.get_cmap('binary'),vmin =-10,vmax=15,
                  transform=ax1.get_transform(wcs))

photomet = []
for i in range(len(regions)):
    pos = SkyCoord(ra=regions[i].center.ra.value, dec=regions[i].center.dec.value, unit='deg')
    aper = SkyEllipticalAperture(pos, a = regions[i].width.value/2*u.arcsec, b= regions[i].height.value/2*u.arcsec,theta=(90+regions[i].angle.value)*u.deg)
    pix = aper.to_pixel(wcs=wcs)
    pix.plot(ax=ax1,fill=False,color='red')
    photomet.append(pix.do_photometry(hdu.data))

print(photomet)

In [ ]:
reg_sums = []
for i in range(len(photomet)):
    reg_sums.append(photomet[i][0][0])
print(reg_sums)
np.savetxt('diffuse_sums.txt',reg_sums, fmt='%s')

In [ ]:
print(photomet[0][0][0])